<a href="https://colab.research.google.com/github/DominikBurkert/PyTorch/blob/master/Transformer_Change_Numbers_basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import output
#I don't know why nbextensions don't work with python3.6 
with output.temporary():
  !pip install --upgrade git+https://github.com/Kreijstal/colab_inspector.git 
  !python2.7 -m pip install --upgrade git+https://github.com/blois/colab_inspector.git #yes, really.
  !jupyter nbextension install --py inspector

import inspector

# open a scratch cell (Ctrl+Alt+N)
# run there
# inspector.watch_globals()

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd


# Daten laden

In [ ]:
from google.colab import files #daten werden nur für die aktuelle sizung geladen!
uploaded = files.upload()

Saving input_data.xlsx to input_data.xlsx
Saving target_data.xlsx to target_data.xlsx


In [ ]:
"""Inputdaten für das Netz"""
# Das sind die Beispiele aus der Transformer Doku
# https://pytorch.org/docs/stable/generated/torch.nn.Transformer.html

#input = torch.rand((10, 32, 512))
#target = torch.rand((20, 32, 512))

"""Input-Daten laden"""
input = pd.read_excel('input_data.xlsx')
input = torch.tensor(input.values)
#transformer benötigt eine 3dimensionale matrix als input
print(input.size())
input = input.unsqueeze(0)
print(input.size())
#input = torch.Tensor([input.values for _ in range(10)]) #damit hat man dim -> input (10,6,10)

""" Target-Daten (diese Daten sollen am Ende rauskommen -> hier: alle Zahlen einmal drehen. aus 0->1, aus 1->0) """
target = pd.read_excel('target_data.xlsx')
target = torch.tensor(target.values)
#natürlich muss auch beim target datensatz eine dimension hinzugefügt werden
print(target.size())
target = target.unsqueeze(0)
print(target.size())
#target = torch.Tensor([target.values for _ in range(10)])

print(target)

torch.Size([6, 10])
torch.Size([1, 6, 10])
torch.Size([6, 10])
torch.Size([1, 6, 10])
tensor([[[0, 1, 0, 1, 0, 1, 0, 1, 0, 1],
         [1, 0, 1, 0, 1, 0, 1, 0, 1, 0],
         [1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 1, 1, 1, 1, 1],
         [0, 0, 0, 1, 1, 1, 0, 0, 0, 1],
         [1, 1, 1, 0, 0, 0, 1, 1, 1, 0]]])


In [ ]:
#transformer benötigt die Daten als Float 
target = target.float()
input = input.float()
print(target)

# Model

In [ ]:
model = torch.nn.Transformer(d_model=10, nhead=1, num_encoder_layers=2, 
                     num_decoder_layers=2,  dropout=0.1, 
                     activation='relu', custom_encoder=None, custom_decoder=None)

#wie soll der fehler berechnet werden? (softmax, mean-squared-error(MSELoss), CrossEntropyLoss)
criterion = nn.MSELoss()

# Training

In [ ]:
for i in range(100):  #100 epochen trainieren
  out = model(input, target) #einmal das netz durchlaufen
  loss = criterion(out, target) #vergleich von output (vorhersage des netzes) und target(echte Werte)-> Berechnung des Fehlers
  print(loss)

  model.zero_grad()
  loss.backward()
  optimizer = optim.SGD(model.parameters(), lr=0.10)
  optimizer.step()

# Überprüfung

In [ ]:
print("Vorhersage: \n", torch.round(out[0,5]))
print("Ziel: \n", target[0,5])

Vorhersage: 
 tensor([1., 1., 1., -0., -0., 0., 1., 1., 1., -0.], grad_fn=<RoundBackward>)
Ziel: 
 tensor([1., 1., 1., 0., 0., 0., 1., 1., 1., 0.])
